In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

In [ ]:
from math import cos, sin, radians
import serial
import time
import cv2
from PIL import Image
from pathlib import Path, PurePath
from datetime import date

In [ ]:
def get_pixels_xy(rings={7: 0, 16: 0, 24: 0}, clockwise=True, cm=True):
    """Calculates the (x, y) coordinates of each Pixel of the chain.
    Returns the coordinates within a list for each configuration.
    
    Args:
        rings: A dictionary (dict) in which the key (int) of the
            key-value pair is the number of Pixels of the configuration
            and the value (float) is the angle in degrees measured from
            the x-axis to the first Pixel of the NeoPixel shape.
        clockwise: (boolean) If it is True, the lighting sequence is
            clockwise.
        cm: (boolean) If is is True, the centimetre is the default
            length unit to calculate the (x, y) coordinates. If cm is
            False, the inch (1 inch = 2,54 cm) is the default unit.
    
    Returns:
        A dictionary (dict) in which the values are list of coordinates
        (tuple). For example
        
        >>> get_pixels_xy({7: -30})
        {'7': [(0, 0), (0.69, -0.4), (0.0, -0.8), (-0.69, -0.4),
               (-0.69, 0.4), (-0.0, 0.8), (0.69, 0.4)]}
    """
    # The properties for each NeoPixel ring or Jewel were retrieved from
    # https://learn.adafruit.com/adafruit-neopixel-uberguide/downloads
    NEOPIXEL_LED_RINGS = {
        '7': {
            'name': 'NeoPixel Jewel',
            'diameter': {
                'outer': 0.9,
                'px': 0.63
            },
            'type': 'jewel'
        },
        '12': {
            'name': 'NeoPixel 12-LED Ring',
            'diameter': {
                'inner': 0.92,
                'outer': 1.45,
                'px': 1.16
            },
            'type': 'ring'
        },
        '16': {
            'name': 'NeoPixel 16-LED Ring',
            'diameter': {
                'inner': 1.25,
                'outer': 1.75,
                'px': 1.49
            },
            'type': 'ring'
        },
        '24': {
            'name': 'NeoPixel 24-LED Ring',
            'diameter': {
                'inner': 2.06,
                'outer': 2.58,
                'px': 2.3
            },
            'type': 'ring'
        }
    }
    INCH_CM = 2.54

    pixels_xy = dict()
    for key in rings:
        try:
            print('<RingValid-' + str(key) + '>')
            ring = NEOPIXEL_LED_RINGS[str(key)]
            angle = 0
            ring_xy_list = list()
            outer_pixels = int(key)
            if ring['type'] == 'jewel':
                ring_xy_list.append((0, 0))  # Add the central Pixel.
                outer_pixels -= 1
            dist_px_px = 360 / float(outer_pixels)  # In degrees.
            phase = float(rings[key])
            radius = ring['diameter']['px'] / 2
            for i in range(outer_pixels):
                pixel_x = cos(radians(angle + phase)) * radius
                pixel_y = sin(radians(angle + phase)) * radius
                if cm:
                    pixel_x = INCH_CM * pixel_x
                    pixel_y = INCH_CM * pixel_y
                xy = tuple([round(pixel_x, 2), round(pixel_y, 2)])
                if clockwise:
                    angle -= dist_px_px
                else:
                    angle += dist_px_px
                ring_xy_list.append(xy)
            pixels_xy[str(key)] = ring_xy_list
        except KeyError:
            print('<RingInvalid-' + str(key) + '>')
    return pixels_xy

In [ ]:
def take_photos(cam, n_photos, color='R', baud=9600, brt=225,
                cam_wait=0, img_prefix='IMG_', img_format='TIF',
                outpath=''):
    """Illuminates a sample under a microscope with different incident
    angle from a NeoPixel illuminator connected to an Arduino and takes
    a photo for each illumination.
    
    Args:
        cam: (int) Video device. Run ls -ltrh /dev/video* to list the
            video devices picked up by the Linux kernel.
        n_photos: (int) Number of photos. The first photo is taken using
            the first Pixel of the lighting sequence, the second photo
            is taken with the second one, and so on.
        color: (string) Color to write: 'r', 'g', 'b' or 'w'.
        baud: (int) Baud rate in bit per seconds for serial data
            transmission.
        brt: (int) Brightness level of the all Pixels. The minimun value
            is 0 and the maximun is 255. 
        cam_wait: (float) Wait time in second between the photo capture
            and the turning off of the current Pixel.
        img_prefix: (string) Common prefix for photo's filename.
        img_format: (string) File format for the photo file. The
            extension file is automatically generated from this value.
        outpath: (string) The directory's path to store the photos.
    
    Returns:
        The path of the direcory (string) that stores the photos.
    
    Raises:
        KeyboardInterrupt: when the execution of this function is
            interrupted or finishing the photo capture process.
        serial.SerialException: when an Arduino is not discovered.
        NameError: when the microscope's camare is not found or is not
            available.
        
    """
    # Pixel WS2812's datasheet for wavelenghts:
    # http://www.seeedstudio.com/document/pdf/WS2812B%20Datasheet.pdf
    color = color.strip().lower()
    if color == 'g':
        # Wavelenght (nm): [515-530]
        color_name = 'Green'
    elif color == 'b':
        # Wavelenght (nm): [465, 475]
        color_name = 'Blue'
    elif color == 'w':
        color_name = 'White'
    else:
        # Wavelength (nm): [620, 630]
        color = 'r'
        color_name = 'Red'
    img_prefix += color.upper()
    this_project_path = None
    # List the serial ports.
    serial_ports = !python -m serial.tools.list_ports
    for i in range(len(serial_ports)):
        # For each line of the output, remove the whitespace characters at
        # the beginning and the end of the string.
        serial_ports[i] = serial_ports[i].strip()
    for item in serial_ports:
        if "ports found" not in item.lower():
            try:
                ser = serial.Serial(item, baud)
                print('<DeviceFound-' + item + '>')
                print('<PC-ArduinoReset>')
                # http://forum.arduino.cc/index.php?topic=38981.0
                # https://stackoverflow.com/questions/21073086/wait-on-arduino-auto-reset-using-pyserial
                ser.dtr = False
                time.sleep(1)
                ser.reset_input_buffer()
                ser.dtr = True
                break
            except serial.SerialException:
                print('<DeviceNotFound>')
    cap = cv2.VideoCapture(cam)
    try:
        if not cap.isOpened():
            # Try to open the cam again.
            if not cap.open(cam):
                # Force to occurr NameError exception is the cam was not
                # opened.
                print('<CamNotFound-' + str(cam) + '>')
                raise NameError
        else:
            print('<CamFound-' + str(cam) + '>')
        ser
        # Make the project's directory.
        home_path = Path.home()
        today_date = str(date.today())
        c = 1
        if outpath != '':
            parent_path = Path(outpath)
        else:
            parent_path = PurePath(home_path, 'SuperScanner', 'projects',
                                   today_date)
        while True:
            this_project_path = PurePath(parent_path, str(c)).as_posix()
            try:
                Path(this_project_path).mkdir(parents=True)
                print('<DirectoryCreated>')
                print(this_project_path)
                break
            except FileExistsError:
                c = c + 1
        while True:
            try:
                line = ser.readline().strip()
                if b'<ArduinoReady>' in line:
                    print(line.decode())
                    ser.write(b'useseq')
                    print('<PC-CustomSeqUse>')
                    while ser.in_waiting == 0:
                        line = ser.readline().strip()
                        if line == b'<CustomSeqUse>':
                            print('<Arduino-CustomSeqUse>')
                            break
                    for i in range(n_photos):
                        outcoming_string = (str(i + 1) + ','
                                            + str(brt) + ',' + color)
                        ser.write(outcoming_string.encode())
                        print('<PC-Pixel' + color_name + 'On-' +
                              str(i + 1) + '>')
                        pixel_on_ev_segment = ('<Arduino-Pixel'+
                                               color_name + 'On-')
                        # https://stackoverflow.com/questions/38645060/what-is-the-equivalent-of-serial-available-in-pyserial
                        while ser.in_waiting == 0:
                            line = ser.readline().strip()
                            if pixel_on_ev_segment.encode() in line:
                                print(line.decode())
                                break
                        ret, frame = cap.read()
                        img_idx = str(i + 1)
                        for j in range((len(str(n_photos)) -
                                        len(str(i + 1)))):
                            img_idx = '0' + img_idx
                        filename = (img_prefix + img_idx +  '.' +
                                    img_format.lower())
                        file_path = this_project_path + '/' + filename
                        # Covert the image to grayscale. OpenCV uses BGR.
                        gray_frame = cv2.cvtColor(frame,
                                                  cv2.COLOR_BGR2GRAY)
                        gray_img = Image.fromarray(gray_frame, mode='L')
                        print('<' + img_format + '-Img'
                              + color_name + 'Write-' + img_idx + '>')
                        gray_img.save(file_path)
                        # Ensure that the photo was taken with light.
                        time.sleep(cam_wait)
                        # Turn the current Pixel off. The color to write
                        # does not matter.
                        outcoming_string = str(i + 1) + ',0,w'
                        ser.write(outcoming_string.encode())
                        print('<PC-PixelOff-' + str(i + 1) + '>')
                        pixel_off_ev_segment = b'<Arduino-PixelOff-'
                        while ser.in_waiting == 0:
                            line = ser.readline().strip()
                            if pixel_off_ev_segment in line:
                                print(line.decode())
                                break
                    raise KeyboardInterrupt
                    break
            except KeyboardInterrupt:
                # https://stackoverflow.com/questions/3208566/nested-exceptions
                raise KeyboardInterrupt('1')
            except serial.SerialException:
                cap.release()
                print('<VideoCaptureClose>')
                break
    except NameError:
        print('<MicroscopeNotAvailable>')
    except KeyboardInterrupt:
        ser.close()
        print('<DeviceClose>')
        cap.release()
        print('<VideoCaptureClose>')
    return this_project_path